In [1]:
import pandas as pd
import os

In [2]:
fire_archive = pd.read_csv(r'C:\Users\Kye Manning-Lees\bushfireModel\fire_archive_2017_to_2021.csv')
fire_archive.acq_date = pd.to_datetime(fire_archive.acq_date)
fire_archive.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,-35.872715,137.311401,341.61,0.41,0.45,2017-01-05,414,N,VIIRS,n,1,303.48,3.10,D,0
1,-35.868961,137.306183,334.45,0.41,0.45,2017-01-05,414,N,VIIRS,l,1,304.93,3.80,D,0
2,-35.792042,137.635727,347.64,0.44,0.38,2017-04-05,426,N,VIIRS,n,1,308.87,13.69,D,0
3,-35.787483,137.584457,347.40,0.44,0.38,2017-04-05,426,N,VIIRS,n,1,306.18,12.90,D,0
4,-35.783913,137.583786,353.03,0.44,0.38,2017-04-05,426,N,VIIRS,n,1,306.90,12.90,D,0


In [3]:
fire_archive.confidence.value_counts()

n    3982
h     953
l     472
Name: confidence, dtype: int64

In [4]:
confidence_h = pd.DataFrame(fire_archive.loc[fire_archive['confidence'] == 'h'].groupby('acq_date').count().reset_index()['acq_date'])
confidence_h.columns = ['date']
print(len(confidence_h))
confidence_h.head()

28


,date
0,2017-04-06
1,2017-04-12
2,2017-10-01
3,2017-11-22
4,2018-01-09


In [20]:
def mergeDF_Labels(df, labelFile):
    df.date = pd.to_datetime(df.date)
    df['label'] = df['date'].isin(labelFile['date']).astype(int)
    print(df.label.value_counts())
    df = df.drop(columns=['Unnamed: 0'])
    return df

labelFile = confidence_h

df_2017 = pd.read_csv(r'C:\Users\Kye Manning-Lees\bushfireModel\data_2017.csv')
df_2018 = pd.read_csv(r'C:\Users\Kye Manning-Lees\bushfireModel\data_2018.csv')
df_2019 = pd.read_csv(r'C:\Users\Kye Manning-Lees\bushfireModel\data_2019.csv')
df_2020 = pd.read_csv(r'C:\Users\Kye Manning-Lees\bushfireModel\data_2020.csv')
concatFrames = pd.concat([df_2017, df_2018, df_2019, df_2020])

labelled_data = mergeDF_Labels(concatFrames, labelFile)
len(labelled_data)

0    1433
1      28
Name: label, dtype: int64


1461

In [21]:
labelled_data = labelled_data.reset_index().drop(columns=['index'])
labelled_data

,Divergence,Fraction of cloud cover,Geopotential,Ozone mass mixing ratio,Potential vorticity,Relative humidity,Specific cloud ice water content,Specific cloud liquid water content,Specific humidity,Specific rain water content,Specific snow water content,Temperature,U component of wind,V component of wind,Vertical velocity,Vorticity (relative),date,label
0,0.000028,0.0,477420.642570,0.000004,-0.003444,0.000132,0.0,0.0,0.000004,0.0,0.0,269.965028,-42.726981,-7.730748,-0.002025,-5.312786e-07,2017-01-01,0
1,-0.000009,0.0,477329.534030,0.000004,-0.003443,0.000121,0.0,0.0,0.000004,0.0,0.0,270.996854,-45.692117,-5.332659,-0.000635,1.199110e-06,2017-01-02,0
2,0.000018,0.0,476226.449334,0.000005,-0.003097,0.000155,0.0,0.0,0.000004,0.0,0.0,267.775899,-47.402067,-2.174820,0.000312,1.595173e-05,2017-01-03,0
3,0.000021,0.0,476254.828078,0.000005,-0.002975,0.000149,0.0,0.0,0.000004,0.0,0.0,268.415721,-48.287632,0.453403,0.001502,2.528151e-05,2017-01-04,0
4,-0.000033,0.0,475870.602938,0.000005,-0.002866,0.000135,0.0,0.0,0.000004,0.0,0.0,269.616232,-50.383268,-5.729242,0.000266,2.406464e-05,2017-01-05,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,-0.000018,0.0,474700.096058,0.000004,-0.003277,0.000121,0.0,0.0,0.000004,0.0,0.0,270.811849,-47.138445,-3.356542,-0.000088,1.443259e-05,2020-12-27,0
1457,0.000014,0.0,474770.937599,0.000004,-0.003325,0.000138,0.0,0.0,0.000004,0.0,0.0,269.145490,-48.325478,-4.213930,-0.000054,9.032533e-06,2020-12-28,0
1458,0.000004,0.0,474724.677807,0.000004,-0.003136,0.000141,0.0,0.0,0.000004,0.0,0.0,268.935514,-52.200542,-5.845193,0.000872,1.135247e-05,2020-12-29,0
1459,0.000006,0.0,474173.128975,0.000005,-0.003229,0.000165,0.0,0.0,0.000004,0.0,0.0,267.048160,-51.691104,-0.231027,-0.000739,1.354588e-05,2020-12-30,0


# Write to CSV

In [22]:
output_dir = r"C:\Users\Kye Manning-Lees\bushfireModel"

In [23]:
# write data df to CSV

filename = os.path.join(output_dir, f'labelled_data.csv')
print(f'Writing to {filename}')
labelled_data.to_csv(filename)
print('done')

Writing to C:\Users\Kye Manning-Lees\bushfireModel\labelled_data.csv
done


# Data Description  

The dataset is the combination of ERA5 temporal statistics and NASA fire detection confidence labels.   
ERA5 dataset: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels?tab=form  
NASA dataset: https://firms.modaps.eosdis.nasa.gov/download/  

#### ERA5 Dataset  
The ERA5 dataset contains the following temporal variables:  
- Divergence
- Fraction of cloud cover
- Geopotential
- Ozone mass mixing ratio
- Potential vorticity
- Relative humidity
- Specific cloud ice water content
- Specific cloud liquid water content
- Specific humidity
- Specific rain water content
- Specific snow water content
- Temperature
- U component of wind
- V component of wind
- Vertical velocity
- Vorticity (relative)  
As well as the following location variables:
- latitude
- longtitude
- time

The latitude and longtitude was removed from the dataset to transform the data into a two dimensional dataframe. Time was translated into a date format in datetime type.   
Temporal variables were averaged across latitude and longtitude, grouped by date. This is only possible because the latitude and longtitude area covered is very small, covering only Kangaroo island.  

#### NASA Dataset
The NASA dataset contains the following temporal variables:  
- brightness (VIIRS I-4 channel brightness temperature of the fire pixel measured in Kelvin)
- scan (The algorithm produces approximately 375 m pixels at nadir. Scan and track reflect actual pixel size.)
- track (The algorithm produces approximately 375 m pixels at nadir. Scan and track reflect actual pixel size.)
- satellite	(N= Suomi National Polar-orbiting Partnership (Suomi NPP))
- instrument	
- confidence (l = low confidence, n = nominal, h = high)
- version ("1.0NRT" - Collection 1 NRT processing, "1.0" - Collection 1 Standard processing.)
- bright_t31 (Channel brightness temperature of the fire pixel measured in Kelvin.)
- frp (Fire Radiative Power)
- daynight	
- type (Inferred hot spot type:0 = presumed vegetation fire, 1 = active volcano, 2 = other static land source, 3 = offshore detection (includes detections over water))
As well as the following location variables:    
- latitude (Center of nominal 375 m fire pixel)
- longitude (Center of nominal 375 m fire pixel)
- acq_date (Acquisition Date)	
- acq_time	(Acquisition Time)

The only variables utilised in the NASA dataset was the confidence and acquisition date.  
I should note that the NASA data only reports when the satellite can see the specified location as it orbits. This results in 'dark' spots, where dates are missing without reported confidence levels of fires. 

### Combined Dataset
The combined data of ERA5 temporal statistics and NASA confidence labels culminates into a dataframe spanning from the first of january 2017 to the first of january 2021.   
The NASA data contributing with 28 cases of 'h' (high confidence) vegetation fires. 
The combined dataset contains 1461 rows and 18 columns. The confidence labels replaced with binary representations, with nominal and low replaced with 0 and occuring 1433 times, while high confidence replaced with 1 and occuring 28 times. 

## Data Source and Information
Features explained
https://earthdata.nasa.gov/earth-observation-data/near-real-time/firms/v1-vnp14imgt#ed-viirs-375m-attributes  

data source: https://earthdata.nasa.gov/learn/articles/feature-articles/wildfire-articles/wildfires-cant-hide-from-earth-observing-satellites

In [24]:
labelled_data.describe()

,Divergence,Fraction of cloud cover,Geopotential,Ozone mass mixing ratio,Potential vorticity,Relative humidity,Specific cloud ice water content,Specific cloud liquid water content,Specific humidity,Specific rain water content,Specific snow water content,Temperature,U component of wind,V component of wind,Vertical velocity,Vorticity (relative),label
count,1.461000e+03,1461.0,1461.000000,1.461000e+03,1461.000000,1461.000000,1461.0,1461.0,1.461000e+03,1461.0,1461.0,1461.000000,1461.000000,1461.000000,1461.000000,1461.000000,1461.000000
mean,-8.983134e-07,0.0,467898.325569,4.953499e-06,-0.003635,0.000496,0.0,0.0,3.868104e-06,0.0,0.0,260.333744,17.495789,-3.094829,-0.000098,0.000007,0.019165
std,1.870612e-05,0.0,6502.303238,5.418807e-07,0.001429,0.000740,0.0,0.0,1.047480e-07,0.0,0.0,9.094679,46.839003,11.224582,0.000681,0.000026,0.137152
min,-7.789531e-05,0.0,447097.774637,3.568890e-06,-0.015553,0.000065,0.0,0.0,3.526385e-06,0.0,0.0,224.229627,-62.059979,-63.020726,-0.003530,-0.000185,0.000000
25%,-1.223198e-05,0.0,462723.055600,4.564839e-06,-0.004271,0.000159,0.0,0.0,3.802994e-06,0.0,0.0,254.060909,-20.695667,-7.516243,-0.000515,-0.000006,0.000000
50%,-8.938646e-07,0.0,468427.574533,4.847106e-06,-0.003486,0.000245,0.0,0.0,3.887761e-06,0.0,0.0,262.234963,14.973556,-3.642041,-0.000103,0.000009,0.000000
75%,1.066576e-05,0.0,473959.424213,5.265374e-06,-0.002924,0.000554,0.0,0.0,3.944394e-06,0.0,0.0,267.494204,51.250119,0.708649,0.000278,0.000021,0.000000
max,1.331420e-04,0.0,479361.872876,7.109981e-06,0.001436,0.013962,0.0,0.0,4.108929e-06,0.0,0.0,280.058843,140.627871,55.211072,0.003796,0.000121,1.000000


In [26]:
len(labelled_data)

1461

In [1]:
# 160+